# A notebook to perform QC on the PIPS T and RH/Td observations

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.stats.mstats import zscore
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from cycler import cycler
import warnings
warnings.simplefilter('ignore')
%matplotlib notebook

In [ ]:
# plt.style.use('seaborn-v0_8-bright')

In [ ]:
# Read in the original PIPS netcdf files

# PIPS_input_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/'
# PIPS_output_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'

# PIPS_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2023/PIPS_data/'
PIPS_base_dir = '/Users/dawson29/Dropbox/Projects/PERiLS/obsdata/2023/'
# PIPS_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2023/PIPS_data/'

deployment_name = 'IOP4_033123' # '031623_mass_test' # 'IOP2_030323' # 'IOP5_040523' # 'IOP4_033123' # 'IOP3_032423' # 'IOP3_040522'
PIPS_input_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf')
PIPS_output_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf_T_Td_QC')
if not os.path.exists(PIPS_output_dir):
    os.makedirs(PIPS_output_dir)

# IOP1 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B']
# IOP2 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
# IOP3 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
# IOP3 2023
# PIPS_names = ['PIPS2A', 'PIPS3A']
# IOP2, IOP4 or IOP5 2023
PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
parsivel_interval = 60
parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_input_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]
output_parsivel_filepaths = [os.path.join(PIPS_output_dir, parsivel_filename) 
                             for parsivel_filename in parsivel_filenames]
conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_input_dir, conv_filename) for conv_filename in conv_filenames]
output_conv_filepaths = [os.path.join(PIPS_output_dir, conv_filename) for conv_filename in conv_filenames]
parsivel_ds_dict = {}
conv_ds_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    try:
        parsivel_ds_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    except:
        parsivel_ds_dict[PIPS_name] = None
    conv_ds_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [ ]:
# default_cycler = (cycler(color=['r', 'orange', 'y', 'g', 'b', 'purple']))

# plt.rc('lines', linewidth=4)
# plt.rc('axes', prop_cycle=default_cycler)

In [ ]:
PIPS_to_comp = ['PIPS3A', 'PIPS3B']

start_times = []
stop_times = []
for PIPS_name in PIPS_to_comp:
    start_times.append(conv_ds_dict[PIPS_name]['time'][0])
    stop_times.append(conv_ds_dict[PIPS_name]['time'][-1])
    
min_time = min(start_times)
max_time = max(stop_times)

print(min_time, max_time)

In [ ]:
time_start = min_time # '2023-03-17T8:30:00'
time_stop = max_time # '2023-03-17T9:30:00'

In [ ]:
# Plot the timeseries of slowtemp and fasttemp
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['slowtemp'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_slowT', 
                                                                    ls='None', marker='o', ms=1., alpha=0.5)    
plt.gca().set_prop_cycle(None)

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['fasttemp'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_fastT', 
                                                                    ls='None', marker='x', ms=5., alpha=0.75)
    
ax.legend()

In [ ]:
# It looks like 3A and 3B have wonky slow-temp sensors that read too high. Let's take the differences between the
# fast temp and slow temp obs and find the average difference

diff_T_dict = {}

for PIPS_name in PIPS_to_comp:
    diff_T = conv_ds_dict[PIPS_name]['slowtemp'] - conv_ds_dict[PIPS_name]['fasttemp']
    diff_T_dict[PIPS_name] = diff_T


In [ ]:
# Plot the time series of the slow-fast temp differences

fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    diff_T = diff_T_dict[PIPS_name]
    diff_T.sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_diff_T', 
                                                       ls='None', marker='o', ms=1., alpha=0.5)

ax.legend()

In [ ]:
# Compute the mean differences and then subtract from the original slowtemp, storing in a new corrected slowtemp

for PIPS_name in PIPS_to_comp:
    diff_T = diff_T_dict[PIPS_name]
    mean_diff_T = diff_T.mean().values
    print(mean_diff_T)
    conv_ds_dict[PIPS_name]['slowtemp_corrected'] = conv_ds_dict[PIPS_name]['slowtemp'].copy() - mean_diff_T
    # Add the value subtracted to the attributes so we know that we modified it
    conv_ds_dict[PIPS_name]['slowtemp_corrected'].attrs['bias_subtracted'] = mean_diff_T
    


In [ ]:
# Ok, now plot the bias-corrected slowtemps along with the fasttemps


fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['slowtemp_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_slowT', 
                                                                              ls='None', marker='o', ms=1., alpha=0.5)    
plt.gca().set_prop_cycle(None)

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['fasttemp'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_fastT', 
                                                                    ls='None', marker='x', ms=5., alpha=0.75)
    
ax.legend()

In [ ]:
# Now, we need to recompute the dewpoint and RH_derived using the corrected slowtemps

for PIPS_name in PIPS_to_comp:
    pressure = conv_ds_dict[PIPS_name]['pressure']
    slowtemp = conv_ds_dict[PIPS_name]['slowtemp_corrected']
    fasttemp = conv_ds_dict[PIPS_name]['fasttemp']
    RH = conv_ds_dict[PIPS_name]['RH']
    dewpoint = thermo.calTdfromRH(pressure * 100., slowtemp + 273.15, RH / 100.) - 273.15
    dewpoint.sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_dewpoint', 
                                                         ls='None', marker='o', ms=1., alpha=0.5)
    RH_derived = thermo.calRH(pressure * 100., fasttemp + 273.15, dewpoint + 273.15) * 100.
    
    conv_ds_dict[PIPS_name]['dewpoint_corrected'] = conv_ds_dict[PIPS_name]['dewpoint'].copy()
    conv_ds_dict[PIPS_name]['dewpoint_corrected'].data = dewpoint
    
    conv_ds_dict[PIPS_name]['RH_derived_corrected'] = conv_ds_dict[PIPS_name]['RH_derived'].copy()
    conv_ds_dict[PIPS_name]['RH_derived_corrected'].data = RH_derived


In [ ]:
# Plot the timeseries of RH and RH_derived
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['RH'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_RH', 
                                                              ls='None', marker='o', ms=1., alpha=0.5)    
plt.gca().set_prop_cycle(None)

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['RH_derived_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_RH_derived', 
                                                                    ls='None', marker='x', ms=5., alpha=0.75)
    
ax.legend()

In [ ]:
# Plot the timeseries of dewpoint
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    conv_ds = conv_ds_dict[PIPS_name]
    conv_ds['dewpoint_corrected'].sel(time=slice(time_start, time_stop)).plot(ax=ax, label=f'{PIPS_name}_dewpoint'), 

ax.legend()

In [ ]:
# Now resample the corrected slowtemp, dewpoint, and RH_derived to the parsivel times

intervalstr = '60S'

for PIPS_name in PIPS_to_comp:
    PSD_datetimes = pips.get_PSD_datetimes(parsivel_ds_dict[PIPS_name]['VD_matrix'])
    sec_offset = PSD_datetimes[0].second
    print(sec_offset)
    offset_str = pips.get_interval_str(sec_offset)
    
    slowtemp_corrected = conv_ds_dict[PIPS_name]['slowtemp_corrected']
    new_slowtemp = slowtemp_corrected.resample(time=intervalstr, label='right', closed='right', 
                                               offset=offset_str).mean()
    dewpoint_corrected = conv_ds_dict[PIPS_name]['dewpoint_corrected']
    new_dewpoint = dewpoint_corrected.resample(time=intervalstr, label='right', closed='right', 
                                               offset=offset_str).mean()
    RH_derived_corrected = conv_ds_dict[PIPS_name]['RH_derived_corrected']
    new_RH_derived = RH_derived_corrected.resample(time=intervalstr, label='right', closed='right', 
                                                   offset=offset_str).mean()
    
    parsivel_ds_dict[PIPS_name]['slowtemp_corrected'] = new_slowtemp
    parsivel_ds_dict[PIPS_name]['dewpoint_corrected'] = new_dewpoint
    parsivel_ds_dict[PIPS_name]['RH_derived_corrected'] = new_RH_derived


In [ ]:
fig, ax = plt.subplots()

for PIPS_name in PIPS_to_comp:
    parsivel_ds_dict[PIPS_name]['RH_derived_corrected'].plot(ax=ax, label=f'{PIPS_name}_RH_derived_corrected', 
                                                             ls='None', marker='o', ms=1., alpha=0.75)
    parsivel_ds_dict[PIPS_name]['RH_derived'].plot(ax=ax, label=f'{PIPS_name}_RH_derived', 
                                                   ls='None', marker='x', ms=5., alpha=0.5)
    
ax.legend()

In [ ]:
# Now save to new output directory
for PIPS_name, output_parsivel_filepath, output_conv_filepath in zip(PIPS_names, 
                                                                     output_parsivel_filepaths, 
                                                                     output_conv_filepaths):
    if PIPS_name in PIPS_to_comp:
        print(PIPS_name)
        
        
        print("Saving {}".format(output_parsivel_filepath))
        parsivel_ds_dict[PIPS_name].to_netcdf(output_parsivel_filepath)
        print("Saving {}".format(output_conv_filepath))
        conv_ds_dict[PIPS_name].to_netcdf(output_conv_filepath)